# Importing packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# I. Centering step

### Functions for gradient and hessian computation

In [20]:
def compute_gradient_ft(t, x, Q, p, A ,b):
    # t the scalar that charachterises the functionn we are trying to minimise
    # x of dimension (n), the point in which we are evaluating the function
    # Q of dimension (n,n)
    # p of dimenson (n)
    # A of dimension (m,n)
    # b of dimension (m)

    m = b.size
    n = x.size
    left_side = t*(2*Q @ x + p)
    right_side = np.zeros(n)
    for i in range (m):
    #     print((b[i] - A[i] @ x).shape)
    #     print(f"A shape: {A.shape}")
    #     print(f"x shape: {x.shape}")
    #     print(f"b shape: {b.shape}")
        right_side += A[i]  / (b[i] - A[i] @ x)



    return left_side + right_side 

def compute_hessian_ft(t, x, Q, A, b):
    # t the scalar that charachterises the functionn we are trying to minimise
    # x of dimension (n), the point in which we are evaluating the function
    # Q of dimension (n,n)
    # A of dimension (m,n)
    # b of dimension (m)
    
    m = b.size
    n = x.size
    left_side = 2 * t * Q
    right_side = np.zeros((n,n))
    for i in range(m):
        right_side += (np.outer(A[i], A[i]) ) / (b[i] - A[i]  @ x)**2

    return left_side + right_side


# n = 10
# m = 4
# t = 1
# x = np.random.randn(n)
# Q = np.random.randn(n,n)
# p = np.random.randn(n)
# A = np.random.randn(m,n)
# b = np.random.randn(m)

# compute_gradient_ft(t, x, Q, p, A ,b)
# compute_hessian_ft(t, x, Q, A, b)


In [21]:
# def ft(x):
#     s = b - A @ x           
#     if np.any(s <= 0):  # value at +infinity if out of bounds
#         return np.inf

#     right_side = t * (x @ Q @ x + p @ x)
#     left_side  = -np.sum(np.log(s))
#     return right_side + left_side

# ft(x)

### Additonal utility functions

In [22]:
def compute_delta_x(grad, hessian):
    return np.linalg.solve(hessian, -grad)

def backtracking(ft, x, delta_x, grad, alpha, beta): # returns the optimal x considering backtracking, not the optimal step
    # x of shape (n)
    # delta_x of shape (n)
    # grad of shape (n)
    step = 1
    while ft(x + step * delta_x) >= ft(x) + alpha * step * grad @ delta_x:
        step = beta * step
    print(step)
    return x + step * delta_x


In [23]:
# n = 10
# m = 4
# t = 1

# x = np.random.randn(n)        
# Q  = np.random.randn(n, n)
# A  = np.random.randn(m, n)

# slack = np.random.rand(m) + 0.5  # strictly positive slack 
# b = A @ x + slack               # so that b - A x0 = slack > 0

# alpha = 0.3
# beta = 0.5

# grad = compute_gradient_ft(t, x, Q, p, A ,b)
# hessian = compute_hessian_ft(t, x, Q, A, b)
# delta_x = compute_delta_x(grad, hessian)

# backtracking(ft, x, delta_x, grad, alpha, beta)




In [24]:
def newton_dercrement(grad, delta_x, eps):
    return - grad @ delta_x/ 2 <= eps

def compute_delta_x(grad, hessian):
    return np.linalg.solve(hessian, -grad)

def backtracking(ft, x, delta_x, grad, alpha, beta): # returns the optimal x considering backtracking, not the optimal step
    # x of shape (n)
    # delta_x of shape (n)
    # grad of shape (n)
    step = 1
    while ft(x + step * delta_x) >= ft(x) + alpha * step * grad @ delta_x:
        step = beta * step
    return x + step * delta_x



def centering_step(Q, p, A, b, t, v0, eps):
    v_running = v0
    list_v_points = []
    while True:
        grad = compute_gradient_ft(t, v_running, Q, p, A ,b)
        hessian = compute_hessian_ft(t, v_running, Q, A, b)
        delta_v = compute_delta_x(grad, hessian)

        if newton_dercrement(grad, delta_v, eps):
            break

        
        def ft(x):
            s = b - A @ x           
            if np.any(s <= 0):  # value at +infinity if out of bounds
                return np.inf

            right_side = t * (x @ Q @ x + p @ x)
            left_side  = -np.sum(np.log(s))
            return right_side + left_side
        
        alpha = 0.3
        beta = 0.5
        v_running = backtracking(ft, v_running, delta_v, grad, alpha, beta)
        list_v_points.append(v_running)
    return list_v_points

# # dimensions
# n = 10
# d = 2

# #LASSO hyperprameter
# lambda_val = 10

# # primal matrix sampling
# X = np.random.randn(n, d)
# y = np.random.randn(n)

# # dual variables reformating
# Q  = np.eye(n) / 2
# p = -y
# A = np.concatenate([X.T, - X.T], axis=0)
# b = lambda_val * np.ones(2 * d)

# # interior points and Newton hyperparametres
# alpha = 0.3
# beta = 0.5
# v0 = np.zeros(n)
# eps_newton = 0.0001

# t = 1
# return_array = centering_step(Q, p, A, b, t, v0, eps_newton)


### Centring step function

In [25]:

def centering_step(Q, p, A, b, t, v0, eps):
    v_running = v0
    list_v_points = []
    while True:
        grad = compute_gradient_ft(t, v_running, Q, p, A ,b)
        hessian = compute_hessian_ft(t, v_running, Q, A, b)
        delta_v = compute_delta_x(grad, hessian)

        if newton_dercrement(grad, delta_v, eps):
            break

        
        def ft(x):
            s = b - A @ x           
            if np.any(s <= 0):  # value at +infinity if out of bounds
                return np.inf

            right_side = t * (x @ Q @ x + p @ x)
            left_side  = -np.sum(np.log(s))
            return right_side + left_side
        
        alpha = 0.3
        beta = 0.5
        v_running = backtracking(ft, v_running, delta_v, grad, alpha, beta)
        list_v_points.append(v_running)
    return list_v_points

# dimensions
n = 10
d = 2

#LASSO hyperprameter
lambda_val = 10

# primal matrix sampling
X = np.random.randn(n, d)
y = np.random.randn(n)

# dual variables reformating
Q  = np.eye(n) / 2
p = -y
A = np.concatenate([X.T, - X.T], axis=0)
b = lambda_val * np.ones(2 * d)

# interior points and Newton hyperparametres
v0 = np.zeros(n)
eps_newton = 0.01

t = 1
return_array = centering_step(Q, p, A, b, t, v0, eps_newton)


# Log-barrier method (interior points)

In [34]:
def barr_method(Q, p, A, b, v0, eps_barr=0.1):
    mu = 1.2
    m = b.size
    eps_newton = 0.1
    t = 1

    list_v_barr = [v0]
    v_running = centering_step(Q, p, A, b, t, v0, eps_newton)[-1]
    list_v_barr.append(v_running)

    while eps_barr <= m / t:
        t = mu * t
        v_running = centering_step(Q, p, A, b, t, v0, eps_newton)[-1]
        list_v_barr.append(v_running)

    return list_v_barr
    
# dimensions
n = 10
d = 2

#LASSO hyperprameter
lambda_val = 10

# primal matrix sampling
X = np.random.randn(n, d)
y = np.random.randn(n)

# dual variables reformating
Q  = np.eye(n) / 2
p = -y
A = np.concatenate([X.T, - X.T], axis=0)
b = lambda_val * np.ones(2 * d)

# interior points and Newton hyperparametres
v0 = np.zeros(n)
eps_barr = 0.1

list_v_barr = barr_method(Q, p, A, b, v0, eps_barr)
print(list_v_barr)

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([-0.07200455, -0.80944024,  0.94223213,  1.22567984,  0.09272208,
       -1.02537487,  0.42616795,  0.12875769,  0.48356805, -0.00638448]), array([-0.07081981, -0.81611026,  0.93788091,  1.2311252 ,  0.08356669,
       -1.04017682,  0.43547543,  0.12575417,  0.48598376, -0.0055803 ]), array([-0.06990386, -0.82192398,  0.93419508,  1.23575263,  0.07552315,
       -1.0529092 ,  0.44362794,  0.12301718,  0.48800669, -0.00475414]), array([-0.06919838, -0.82695932,  0.93108311,  1.23967107,  0.06850874,
       -1.06380947,  0.45071901,  0.12055703,  0.48969657, -0.00394434]), array([-0.06865605, -0.83129623,  0.92846256,  1.24297948,  0.06243173,
       -1.07310306,  0.45684884,  0.11837149,  0.49110579, -0.00317679]), array([-0.06823926, -0.8350134 ,  0.92626044,  1.2457662 ,  0.05719695,
       -1.08099891,  0.46211918,  0.11644923,  0.49227962, -0.00246736]), array([-0.06791858, -0.83818594,  0.92441304,  1.24810893,  0.0527101 ,
  

In [35]:
import cvxpy as cp
import numpy as np

def solve_qp(Q, p, A, b):
    x = cp.Variable(Q.shape[0])
    prob = cp.Problem(cp.Minimize(cp.quad_form(x, Q) + p @ x), [A @ x <= b])
    prob.solve()
    return x.value, prob.value

solve_qp(Q, p, A, b)

Polishing not needed - no active set detected at optimal point


(array([-0.06674909, -0.85517207,  0.9150265 ,  1.26009679,  0.02839281,
        -1.12270046,  0.49096218,  0.10524088,  0.4981009 ,  0.00220542]),
 np.float64(-2.4611833797718914))

In [36]:
def f(x, Q, p): 
    return x @ Q @ x + p @ x

f(list_v_barr[-1], Q, p)

np.float64(-2.4611762731416253)

In [37]:
list_v_barr[-1]

array([-0.06679922, -0.85400709,  0.91563937,  1.25930317,  0.03007345,
       -1.12034396,  0.4892851 ,  0.10592342,  0.49778906,  0.00189766])